## Dynamic nanoscale architecture of synaptic vesicle fusion in mouse hippocampal neurons
### Mesoscopic simulation of synaptic vesicle docking -- part 3: Markov State Model

In [ ]:
import numpy as np
from utilities import visualization as vis
from scipy.optimize import minimize

In [ ]:
def build_transition_matrix(chain_transition_probs, template_matrix):
    """
    Constructs the transition matrix based on a sparse template matrix
    """

    assert template_matrix.shape[0] == template_matrix.shape[1]
    N = template_matrix.shape[0]

    assert len(chain_transition_probs) == np.sum(template_matrix) - N
    
    P = np.zeros((N, N))

    current_index = 0
    
    for i in range(N):
        last_index_in_row = 0
        for j in range(N):
            if template_matrix[i, j] > 0:
                last_index_in_row = j
        
        for j in range(last_index_in_row):
            if template_matrix[i, j] > 0:
                P[i, j] = chain_transition_probs[current_index]
                
                current_index += 1

        total_sum_of_row = np.sum(P[i, :])
        
        if total_sum_of_row > 1.0:
            P[i, :] = 0.999 * P[i, :] / total_sum_of_row
            total_sum_of_row = np.sum(P[i, :])
            
        P[i, last_index_in_row] = 1.0 - total_sum_of_row

    return P

In [ ]:
def objective(chain_transition_probs, template_matrix, pi_0, pi_t_observed, t):
    
    P = build_transition_matrix(chain_transition_probs, template_matrix)
    
    P_t = np.linalg.matrix_power(P, t)
    pi_t_pred = pi_0 @ P_t
    
    return np.linalg.norm(pi_t_pred - pi_t_observed)

In [ ]:
def fit_chain_markov_model(pi_t_observed, template_matrix, t, optimization_target=5.0e-9):

    assert template_matrix.shape[0] == template_matrix.shape[1]

    N = template_matrix.shape[0]
    
    # N conditions on sum of the rows
    n_variables = np.sum(template_matrix) - N
    
    pi_0 = np.zeros(N)
    pi_0[0] = 1.0  # All start in state 1

    min_fun = 1e10

    i = 0

    x0 = np.zeros(n_variables)
    
    while min_fun > optimization_target:

        current_index = 0
        
        for j in range(N):
            n_variables_in_row = np.sum(template_matrix[j, :]) - 1
            
            x0[current_index:current_index + n_variables_in_row] = 0.9 * np.random.random_sample(n_variables_in_row) / float(n_variables_in_row + 1)

            current_index += n_variables_in_row
    
        # Bounds: transition probs between 0 and 1
        bounds = [(0.0, 1.0)] * n_variables
    
        result = minimize(objective, x0, args=(template_matrix, pi_0, pi_t_observed, t),
                          bounds=bounds, tol=1.0e-12, options={'maxiter' : 1000000})
        
        if not result.success:
            continue
        else:
            i += 1
            if i % 100 == 0:
                print(f"iteration: {i} finished successfully! function value = {result.fun}")

        if result.fun < min_fun:
            min_fun = result.fun
            best_transition_probs = result.x
            print(f"current best: {result.fun}")
        
    return build_transition_matrix(best_transition_probs, template_matrix), best_transition_probs
    

Setting global variables:

In [ ]:
# Minimum error acceptable in optimization
min_error = 3.0e-9

total_time = 2.0 #ms
# Let's consider each step to happen in 0.2 ms
dt = 0.2
# Number of steps until total freezing
t = int(total_time / dt)
dt = total_time / t
print(f"t = {t} steps, dt = {dt} ms")

## Case 1: without the tight-docking state

States defined in this case:

 - 1: Invagination
 - 2: Stalk formation
 - 3: Closed fusion pore
 - 4: Open fusion pore
 - 5: Dilating fusion pore
 - 6: Collapsing fusion pore
 - 7: Bump

In [ ]:
# Based on the number of observation of each state
p_observed_1 = np.array([10.0, 22.0, 6.0, 7.0, 8.0, 12.0, 21.0])
p_observed_1 = p_observed_1 / np.sum(p_observed_1)

print(p_observed_1)

Fitting the Markov State Model via minimizing the difference between the predicted and observed probabilities of each state:

In [ ]:
template_matrix = np.array([[1, 1, 0, 0, 0, 0, 0],
                            [1, 1, 1, 0, 0, 0, 0],
                            [0, 0, 1, 1, 0, 0, 0],
                            [0, 0, 0, 1, 1, 0, 0],
                            [0, 0, 0, 0, 1, 1, 0],
                            [0, 0, 0, 0, 0, 1, 1],
                            [0, 0, 0, 0, 0, 0, 1]])

P_fitted_1, chain_transition_probs_1 = fit_chain_markov_model(p_observed_1, template_matrix, t, min_error)

print("Fitted transition matrix:")
print(P_fitted_1)

print("\nEstimated transition probabilities (p_i):")
print(chain_transition_probs_1)

Comparing the predicted and observed probabilities (sanity check on the optimization procedure):

In [ ]:
pi_current = np.zeros_like(p_observed_1)
pi_current[0] = 1.0

pi_list = []

for i in range(t):
    pi_list.append(pi_current.copy())
    pi_current = pi_current @ P_fitted_1

pi_list = np.array(pi_list)

print(pi_current)
print(p_observed_1)

In [ ]:
labels = {0: "1",
          1: "2",
          2: "3",
          3: "4",
          4: "5",
          5: "6",
          6: "7"}

node_color = [vis.hex_to_rgb('#e55d79'),
              vis.hex_to_rgb('#e89096'),
              vis.hex_to_rgb('#f1af98'),
              vis.hex_to_rgb('#f9cc84'),
              vis.hex_to_rgb('#a0cb97'),
              vis.hex_to_rgb('#7abdac'),
              vis.hex_to_rgb('#398ea7')]

fig = vis.visualize_markov_chain(P_fitted_1, dt, use_timescale_instead_of_probabilities=True,
                       labels=labels, node_color=node_color)

## Case 2: With the tight-docking state in between 2 and 3

States defined in this case:

 - 1: Invagination
 - 2: Stalk formation
 - 2': Tight docking
 - 3: Closed fusion pore
 - 4: Open fusion pore
 - 5: Dilating fusion pore
 - 6: Collapsing fusion pore
 - 7: Bump


In [ ]:
p_observed_2 = np.array([10.0, 22.0, 1.0, 6.0, 7.0, 8.0, 12.0, 21.0])
p_observed_2 = p_observed_2 / np.sum(p_observed_2)

print(p_observed_2)

In [ ]:
#----------------------------1  2  2' 3  4  5  6  7
template_matrix = np.array([[1, 1, 0, 0, 0, 0, 0, 0],
                            [1, 1, 1, 1, 0, 0, 0, 0],
                            [0, 0, 1, 1, 0, 0, 0, 0],
                            [0, 0, 0, 1, 1, 0, 0, 0],
                            [0, 0, 0, 0, 1, 1, 0, 0],
                            [0, 0, 0, 0, 0, 1, 1, 0],
                            [0, 0, 0, 0, 0, 0, 1, 1],
                            [0, 0, 0, 0, 0, 0, 0, 1]])

P_fitted_2, chain_transition_probs_2 = fit_chain_markov_model(p_observed_2, template_matrix, t, min_error)

print("Fitted transition matrix:")
print(P_fitted_2)

print("\nEstimated transition probabilities (p_i):")
print(chain_transition_probs_2)

In [ ]:
pi_current = np.zeros_like(p_observed_2)
pi_current[0] = 1.0

pi_list = []

for i in range(t):
    pi_list.append(pi_current.copy())
    pi_current = pi_current @ P_fitted_2

pi_list = np.array(pi_list)

print(pi_current)
print(p_observed_2)

In [ ]:
labels = {0: "1",
          1: "2",
          2: r"2$^\prime$",
          3: "3",
          4: "4",
          5: "5",
          6: "6",
          7: "7"}

node_color = [vis.hex_to_rgb('#e55d79'),
              vis.hex_to_rgb('#e89096'),
              vis.hex_to_rgb('#e89096'),
              vis.hex_to_rgb('#f1af98'),
              vis.hex_to_rgb('#f9cc84'),
              vis.hex_to_rgb('#a0cb97'),
              vis.hex_to_rgb('#7abdac'),
              vis.hex_to_rgb('#398ea7')]

pos = {}

pos[0] = (0, 0)
pos[1] = (1, 0)
pos[2] = (1, -1)

for i in range(3, 8):
    pos[i] = (i - 1, 0)
    
fig = vis.visualize_markov_chain(P_fitted_2, dt, pos, use_timescale_instead_of_probabilities=True,
                       labels=labels, node_color=node_color)

## Case 3: With the tight-docking state in between 1 and 2

States defined in this case:

 - 1: Invagination
 - 1': Tight docking
 - 2: Stalk formation
 - 3: Closed fusion pore
 - 4: Open fusion pore
 - 5: Dilating fusion pore
 - 6: Collapsing fusion pore
 - 7: Bump


In [ ]:
p_observed_3 = np.array([10.0, 1.0, 22.0, 6.0, 7.0, 8.0, 12.0, 21.0])
p_observed_3 = p_observed_3 / np.sum(p_observed_3)

print(p_observed_3)

In [ ]:
#----------------------------1  1' 2  3  4  5  6  7
template_matrix = np.array([[1, 1, 1, 0, 0, 0, 0, 0],  #1
                            [0, 1, 1, 0, 0, 0, 0, 0],  #1'
                            [1, 0, 1, 1, 0, 0, 0, 0],  #2
                            [0, 0, 0, 1, 1, 0, 0, 0],  #3
                            [0, 0, 0, 0, 1, 1, 0, 0],  #4
                            [0, 0, 0, 0, 0, 1, 1, 0],  #5
                            [0, 0, 0, 0, 0, 0, 1, 1],  #6
                            [0, 0, 0, 0, 0, 0, 0, 1]]) #7

P_fitted_3, chain_transition_probs_3 = fit_chain_markov_model(p_observed_3, template_matrix, t, min_error)

print("Fitted transition matrix:")
print(P_fitted_3)

print("\nEstimated transition probabilities (p_i):")
print(chain_transition_probs_3)

In [ ]:
pi_current = np.zeros_like(p_observed_3)
pi_current[0] = 1.0

pi_list = []

for i in range(t):
    pi_list.append(pi_current.copy())
    pi_current = pi_current @ P_fitted_3

pi_list = np.array(pi_list)

print(pi_current)
print(p_observed_3)

In [ ]:
labels = {0: "1",
          1: r"1$^\prime$",
          2: "2",
          3: "3",
          4: "4",
          5: "5",
          6: "6",
          7: "7"}

node_color = [vis.hex_to_rgb('#e55d79'),
              vis.hex_to_rgb('#e55d79'),
              vis.hex_to_rgb('#e89096'),
              vis.hex_to_rgb('#f1af98'),
              vis.hex_to_rgb('#f9cc84'),
              vis.hex_to_rgb('#a0cb97'),
              vis.hex_to_rgb('#7abdac'),
              vis.hex_to_rgb('#398ea7')]

pos = {}

pos[0] = (0, 0)
pos[1] = (0, -1)
for i in range(2, 8):
    pos[i] = (i - 1, 0)
    
fig = vis.visualize_markov_chain(P_fitted_3, dt, pos, use_timescale_instead_of_probabilities=True,
                       labels=labels, node_color=node_color)